In [1]:
import json
import pandas as pd
import numpy as np
import numexpr as ne
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from dotenv import load_dotenv
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import torch.cuda.amp as amp

from src.model.model import save_model, load_inference_model
from src.dataset.df import check_dataset
from src.utils.common import set_seed

from src.experiment.inference import inference
from src.experiment.initialize import init_exp
from src.utils.metrics import compute_surface_dice_score_from_volume
from src.model.model import build_model
from src.model.loss import get_lossfn
from src.utils.metrics import get_metrics
from src.model.scheduler import get_scheduler
from src.dataset.common import get_train_dataset

In [2]:
class cfg:
    debug = False
    check_dataset = False

    # = data CFG ====================================================

    dataset_path = "/kaggle/working/dataset/train02_xy_256_128_z_1_1/"
    train_dataset = "Base2dDataset"
    negative_sample_rate = 0.2

    # = experiment CFG =================================================

    project = "SenNet"
    exp_name = os.path.basename(os.getcwd())
    notes = "training with pseudo label model"

    # = model CFG ======================================================

    model_arch = "Unet"
    backbone = "se_resnext50_32x4d"
    in_chans = 1
    target_size = 1

    # = training CFG ===================================================

    epochs = 30

    train_batch_size = 64
    valid_batch_size = train_batch_size

    loss = "DiceLoss"
    metrics = "Dice"
    lr = 5e-4
    num_workers = 12

    # = augmentation ===================================================

    image_size = 256
    train_aug = [
        A.RandomRotate90(p=0.5),
        A.RandomGamma(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug = [
        ToTensorV2(transpose_mask=True),
    ]

    # =============== inference ========================================

    test_dataset = "BaseInferenceDataset"
    stride = image_size // 2
    drop_egde_pixel = 32


load_dotenv("/kaggle/key.env")
set_seed()

In [3]:
def filter_dataset(df):
    # trainのうちlabelが全くないものは90%の確率で除外
    df["random"] = np.random.rand(len(df))
    df = df[(df["sum"] > 0) | (df["fold0"] == "valid") | (df["random"] < cfg.negative_sample_rate)]
    df = df.reset_index(drop=True)
    df = df.drop(["random"], axis=1)
    return df


df = pd.read_csv(f"{cfg.dataset_path}/dataset.csv")
df = filter_dataset(df)
if cfg.debug:
    df = df.sample(10000).reset_index(drop=True)
display(df)

if cfg.check_dataset:
    check_dataset(df, cfg)

,image_path,label_path,fname,kidney,x,y,z,std,sum,fold0,fold1,fold2,fold3,fold4
0,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x0_y0_z103_std0036_sum0,kidney_1_dense,0,0,103,36,0,train,train,train,valid,train
1,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x0_y0_z104_std0036_sum0,kidney_1_dense,0,0,104,36,0,train,train,train,valid,train
2,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x0_y0_z105_std0036_sum0,kidney_1_dense,0,0,105,36,0,train,train,train,valid,train
3,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x0_y0_z109_std0035_sum0,kidney_1_dense,0,0,109,35,0,train,train,train,valid,train
4,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x0_y0_z112_std0034_sum0,kidney_1_dense,0,0,112,34,0,train,train,train,valid,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455302,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x896_y896_z997_std0148_sum0,kidney_9_pseudo,896,896,997,148,0,valid,train,train,train,train
1455303,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x896_y896_z998_std0147_sum0,kidney_9_pseudo,896,896,998,147,0,valid,train,train,train,train
1455304,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x896_y896_z999_std0145_sum0,kidney_9_pseudo,896,896,999,145,0,valid,train,train,train,train
1455305,/kaggle/working/dataset/train02_xy_256_128_z_1...,/kaggle/working/dataset/train02_xy_256_128_z_1...,x896_y896_z99_std0047_sum0,kidney_9_pseudo,896,896,99,47,0,valid,train,train,train,train


In [4]:
if cfg.debug:
    print("!!!Debug mode!!!\n")
    cfg.epochs = 5

for fold in [4]:
    dataset = get_train_dataset(cfg)

    train_df = df[df[f"fold{fold}"] == "train"]
    valid_df = df[df[f"fold{fold}"] == "valid"]

    train_dataset = dataset(train_df, cfg, is_train=True)
    valid_dataset = dataset(valid_df, cfg, is_train=False)

    train_dataloader = DataLoader(train_dataset, batch_size=cfg.train_batch_size, num_workers=cfg.num_workers, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=cfg.valid_batch_size, num_workers=cfg.num_workers, shuffle=False)

    model = build_model(cfg.model_arch, cfg.backbone, cfg.in_chans, cfg.target_size)
    scaler = torch.cuda.amp.GradScaler()
    criterion = get_lossfn(cfg)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr)
    scheduler = get_scheduler(cfg, optimizer)
    metrics = get_metrics(cfg)

    # model, scaler, criterion, optimizer, scheduler, metrics = init_model(cfg)
    slacknotify = init_exp(fold, cfg)

    path_best = f"./{cfg.exp_name}/{cfg.exp_name}_best_fold{fold}.pth"
    path_last = f"./{cfg.exp_name}/{cfg.exp_name}_last_fold{fold}.pth"

    best_loss = float("inf")
    for epoch in range(cfg.epochs):
        model.train()
        total_loss = 0.0
        pbar_train = tqdm(enumerate(train_dataloader), total=len(train_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-0b}")

        for i, (images, masks) in pbar_train:
            images, masks = images.cuda(), masks.cuda()
            optimizer.zero_grad()

            with amp.autocast():
                preds = model(images)
                loss = criterion(preds, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.detach().item()

            loss_ = total_loss / (i + 1)
            lr = f"LR : {scheduler.get_lr()[0]:.2E}"
            gpu_mem = f"Mem : {torch.cuda.memory_reserved() / 1E9:.3g}GB"
            pbar_train.set_description(("%10s  " * 3 + "%10s") % (f"Epoch {epoch}/{cfg.epochs}", gpu_mem, lr, f"Loss: {loss_:.4f}"))

        train_loss = loss_
        scheduler.step()
        wandb.log({"epoch": epoch, "train_loss": train_loss})

        model.eval()
        total_loss = 0.0
        pbar_val = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}")

        for i, (images, masks) in pbar_val:
            images, masks = images.cuda(), masks.cuda()
            with torch.no_grad():
                preds = model(images)
                loss = criterion(preds, masks)
                total_loss += loss.item()

            loss_ = total_loss / (i + 1)
            pbar_val.set_description(("%10s") % (f"Val Loss: {loss_:.4f}"))
        valid_loss = loss_
        # wandb.log({"epoch": epoch, "valid_loss": valid_loss})

        if valid_loss < best_loss:
            print(f"loss : {valid_loss:.4f}\tSAVED MODEL\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}\tBEST")
            best_loss = valid_loss
            save_model(model, cfg, path_best, loss=loss)
        else:
            print(f"loss : {valid_loss:.4f}\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {valid_loss:.4f}")

    save_model(model, cfg, path_last, loss=valid_loss)
    wandb.config.update({"last_loss": valid_loss, "best_loss": best_loss})

    slacknotify.send_reply(f"{cfg.exp_name}_fold{fold} training finished\nbest loss : {best_loss:.4f} last loss : {loss_:.4f}", True)

    if wandb.run:
        wandb.finish()

model_arch:  Unet
backbone:  se_resnext50_32x4d


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: welshonionman. Use `wandb login --relogin` to force relogin


Epoch 0/30  Mem : 8.95GB  LR : 5.00E-04  Loss: 0.0858:  56%|█████▌    | 12750/22740 [35:05<27:30,  6.05it/s] 


KeyboardInterrupt: 

In [4]:
# valid
fold_dict = json.load(open("/kaggle/working/dataset/stack_train02/fold.json", "r"))
pth_types = ["best"]
for pth_type in pth_types:
    for fold in range(4):
        kidney = fold_dict[f"fold{fold}"]["valid"][0]
        pth_path = f"./{cfg.exp_name}/{cfg.exp_name}_{pth_type}_fold{fold}.pth"
        stack_path = f"/kaggle/working/dataset/stack_train02/{kidney}_images.npy"
        label_path = f"/kaggle/working/dataset/stack_train02/{kidney}_labels.npy"
        preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"
        print(f"fold{fold}")
        print(kidney)
        if not os.path.exists(preds_path):
            model = load_inference_model(pth_path, cfg)

            print(kidney)
            print(stack_path)
            inference(model, stack_path, preds_path, cfg)

        label = np.load(label_path)
        preds = np.load(preds_path)
        thresh_score_dict = {}

        for thresh in np.arange(0.1, 0.99, 0.1):
            thresh = round(thresh, 5)
            thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)
            print(thresh, thresh_score_dict[thresh])

        max_score_thresh = max(thresh_score_dict, key=thresh_score_dict.get)

        for thresh in np.arange(max_score_thresh - 0.1, max_score_thresh + 0.1, 0.01):
            thresh = round(thresh, 5)
            thresh_score_dict[thresh] = compute_surface_dice_score_from_volume(ne.evaluate("preds > thresh"), label)
            print(thresh, thresh_score_dict[thresh])

        print(max(thresh_score_dict.items(), key=lambda x: x[1]))

fold0
kidney_9_pseudo
0.1 0.9008
0.2 0.92
0.3 0.9333
0.4 0.9441
0.5 0.9488
0.6 0.951
0.7 0.9512
0.8 0.9463
0.9 0.9328
0.6 0.951
0.61 0.9512
0.62 0.9513
0.63 0.9514
0.64 0.9514
0.65 0.9515
0.66 0.9515
0.67 0.9516
0.68 0.9515
0.69 0.9513
0.7 0.9512
0.71 0.951
0.72 0.9508
0.73 0.9506
0.74 0.9503
0.75 0.9495
0.76 0.9489
0.77 0.9486
0.78 0.948
0.79 0.9476
(0.67, 0.9516)
fold1
kidney_3_pseudo
0.1 0.8928
0.2 0.9108
0.3 0.9217
0.4 0.9271
0.5 0.9277
0.6 0.925
0.7 0.9072
0.8 0.8895
0.9 0.865
0.4 0.9271
0.41 0.9272
0.42 0.9276
0.43 0.9276
0.44 0.9277
0.45 0.9276
0.46 0.9278
0.47 0.9278
0.48 0.9277
0.49 0.9277
0.5 0.9277
0.51 0.9275
0.52 0.9274
0.53 0.9273
0.54 0.9272
0.55 0.9272
0.56 0.927
0.57 0.927
0.58 0.9268
0.59 0.9253
(0.46, 0.9278)
fold2
kidney_2_pseudo
0.1 0.9084
0.2 0.9147
0.3 0.9178
0.4 0.9172
0.5 0.9115
0.6 0.9041
0.7 0.8954
0.8 0.8848
0.9 0.8672
0.2 0.9147
0.21 0.9153
0.22 0.9156
0.23 0.9159
0.24 0.9161
0.25 0.9167
0.26 0.917
0.27 0.9172
0.28 0.9174
0.29 0.9175
0.3 0.9178
0.31 0.9179


In [3]:
# infer
pth_type = "best"
fold = 2
kidneys = ["kidney_2_sparse", "kidney_3_sparse", "kidney_9_pseudo"]

pth_path = f"./{cfg.exp_name}/{cfg.exp_name}_{pth_type}_fold{fold}.pth"
for kidney in kidneys:
    stack_path = f"/kaggle/working/dataset/stack_train01/{kidney}_images.npy"
    preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"

    model = load_inference_model(pth_path, cfg)

    print(stack_path)
    inference(model, stack_path, preds_path, cfg)

model_name Unet
backbone se_resnext50_32x4d
/kaggle/working/dataset/stack_clipped/kidney_2_sparse_images.npy
././preds/kidney_2_sparse_preds_0.npy


100%|██████████| 2216/2216 [10:13<00:00,  3.61it/s]


././preds/kidney_2_sparse_preds_1.npy


100%|██████████| 1040/1040 [09:36<00:00,  1.80it/s]


././preds/kidney_2_sparse_preds_2.npy


...

100%|██████████| 2140/2140 [10:50<00:00,  3.29it/s]


././preds/kidney_9_pseudo_preds_1.npy


100%|██████████| 1107/1107 [10:19<00:00,  1.79it/s]


././preds/kidney_9_pseudo_preds_2.npy


100%|██████████| 1643/1643 [10:39<00:00,  2.57it/s]


In [15]:
pth_type = "best"
kidneys = ["kidney_2_sparse", "kidney_3_sparse", "kidney_9_pseudo"]
for kidney in kidneys:
    stack_path = f"/kaggle/working/dataset/stack_train01/{kidney}_images.npy"
    preds_path = f"./preds/{kidney}_{pth_type}_preds.npy"
    preds_th_path = f"./preds/{kidney}_{pth_type}_preds_th.npy"
    preds = np.load(preds_path)
    preds_th = (preds > 0.67).astype(np.bool_)
    np.save(preds_th_path, preds > 0.67)

In [1]:
import numpy as np